# Dynamically-determined ouput files

* **Difficulty level**: intermediate
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * `dynamic` output targets are resolved after the step is completed  

### `dynamic` output files

Similar to the cases with [dynamic input files](#dynamically-determined-input-files-function-dynamic), the output of some steps could also not be determined beforehand.

For example, with the following script that generates `.out` files that cannot be determined during dry run,

In [1]:
!rm -rf temp_dynamic

%run -v1
[10]
output: dynamic('temp_dynamic/*.out')

import random
import os

for i in range(5):
    os.makedirs('temp_dynamic', exist_ok=True)
    file_target(f'temp_dynamic/{random.randint(1000, 2000)}.out').touch()

[20]
input: group_by=1
print(f'Processing {_input}')

Processing temp_dynamic/1116.out


Processing temp_dynamic/1245.out


Processing temp_dynamic/1248.out


Processing temp_dynamic/1266.out


Processing temp_dynamic/1274.out


Processing temp_dynamic/1439.out


Processing temp_dynamic/1738.out


Processing temp_dynamic/1742.out


Processing temp_dynamic/1774.out


Processing temp_dynamic/1921.out


In this case, you will need to define the output as `dynamic` using a `dynamic` function.

## Dynamic output from substeps

It is a bit tricky to make dynamic output work with substeps because substeps are supposed to produce distinct outputs, and expressions like `temp_dynamic/*.out` could grab output from multiple substeps. It is therefore recommended that the pattern in `dynamic` output vary among substeps as follows:

In [2]:
!rm -rf temp_dynamic

%run -v1
[10]
input: for_each=dict(i=range(5))
output: dynamic(f'temp_dynamic/{_index}_*.out')

import random
import os

os.makedirs(f'temp_dynamic', exist_ok=True)
file_target(f'temp_dynamic/{_index}_{random.randint(1000, 2000)}.out').touch()

[20]
print(f'Processing {_input}')

Processing temp_dynamic/0_1792.out


Processing temp_dynamic/1_1230.out


Processing temp_dynamic/2_1372.out


Processing temp_dynamic/3_1974.out


Processing temp_dynamic/4_1844.out


## Dynamic expansion of non-existing files

Whereas `dynamic` function is usually used with a filename pattern, it can also be used to fixed filenames, which would be "expanded" to an empty list if the file does not exist.

This fact can be used to handle cases when the output might not be generated for some reason. For example, in the following workflow there is a 50% chance that output from a substep will not be generated. Instead of raising an error, use of `dynamic` around the output generates an empty output.

In [3]:
!rm -rf test_*.out

%run -v1 -s force
[10]
input: for_each=dict(i=range(10))
output: dynamic(f'test_{i}.out')

import random
if random.random() > 0.5:
    file_target(f'test_{i}.out').touch()

[20]
input: 
print(f'Processing {_input}')

Processing test_0.out


Processing test_1.out


Processing test_2.out


Processing 


Processing 


Processing 


Processing 


Processing test_7.out


Processing test_8.out


Processing test_9.out


Note that you can remove empty substeps by regrouping `step_input` in the subsequent processing steps

In [4]:
!rm -rf test_*.out

%run -v1 -s force
[10]
input: for_each=dict(i=range(10))
output: dynamic(f'test_{i}.out')

import random
if random.random() > 0.5:
    file_target(f'test_{i}.out').touch()

[20]
input: group_by=1
print(f'Processing {_input}')

Processing test_1.out


Processing test_2.out


Processing test_5.out


Processing test_6.out


Processing test_8.out


## Further reading

* [`input` statement](input_statement.html)
* [`output` statement](output_statement.html)
* [dynamic input](dynamic_input.html)